# 1. 
Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.


In [1]:
import string
from collections import Counter
import re
import numpy as np
from random import shuffle

In [2]:
LATIN = 'abcdefghijklmnopqrstuvwxyz'
DIACRITIC = 'êéâèûôёîçâíäöüåà'
DIGITS = '0123456789'
PUNCT = '–«—»…„“' + '\t\n' + LATIN + DIACRITIC + DIGITS + string.punctuation
EPS = 1e-6
N_SIMBOLS = 33 # нет буквы ё, но есть пробел

SEED = 42
np.random.seed(SEED)

случайная перестановка символов алфавита: 
`j = code[i]` означает, что исходной букве с номером `i-1` соответствует буква с номером `j-1`

In [3]:
code = np.arange(N_SIMBOLS)  
np.random.shuffle(code)

In [4]:
class CharFreq():
    def __init__(self):
        self.counter = Counter()
        self.count_chars = 0
        self.total_encode_text = []
        self.total_clean_text = []
        
    def _get_count_letters(self, line, is_encode):
        encode_text = ''
        clean_line = ''
        for char in line:
            if char not in PUNCT:
                clean_line += char
                if is_encode:
                    char = encoder(char, code)
                    encode_text += char
                self.counter[char] += 1
                self.count_chars += 1
        if encode_text:
            self.total_encode_text.append(encode_text)
        if clean_line:
            self.total_clean_text.append(clean_line)
                
    def preprocessing(self, line, is_encode=False):
        line = line.lower()
        self._get_count_letters(line, is_encode=is_encode)
        self._sort_chars()
        
    def _sort_chars(self):
        self.counter = Counter({k: v for k, v in sorted(self.counter.items(), key=lambda item: -item[1])})
    

Для подсчета статистики будем использовать русский текст "Войны и Мир"

In [5]:
train = CharFreq()
with open('WarAndPeace.txt', 'r') as f:
    for line in f:
        train.preprocessing(line)

In [6]:
def letter2index(char):
    if char == ' ':
        return ord(char)
    return ord(char) - ord('а')


def index2letter(num):
    if num == 32:
        return ' '
    return chr(num + ord('а'))


def encoder(char, code):
    return index2letter(code[letter2index(char)])


def decoder(char, code):
    return index2letter(list(code).index(letter2index(char)))


def decoding_text(text, code):
    decode_text = ''
    for char in text:
        decode_text += decoder(char, code)
    return decode_text


class MyDecoder():
    def __init__(self, freq_dict_train, freq_dict_test):
        self.freq_dict_train = freq_dict_train
        self.freq_dict_test = freq_dict_test
        self.decode_dict = {}
        self.decode_text = []
        self.freq_decode()
        
    def freq_decode(self):
        for key_tr, key_ts in zip(self.freq_dict_train, self.freq_dict_test):
            self.decode_dict[key_ts] = key_tr

    def get_decode_chars(self, line):
        new_line = ''
        for char in line:
            if char in self.decode_dict:
                new_line += self.decode_dict[char]
            else:
                new_line += char
        return new_line
        
    def get_decode_bigrams(self, line):
        '''Декодируем скользящим окном длиной 2 с шагом 2'''
        new_line = ''
        if len(line) % 2: # если нечетное число символов в строке, добавим в конец пробел
            line += ' '
        for i in range(0, len(line) - 1, 2):
            bigram = line[i] + line[i + 1]
            if bigram in self.decode_dict:
                new_line += self.decode_dict[bigram]
            else:
                new_line += bigram
        return new_line

In [23]:
test = CharFreq()

with open('LionAnd_Dog.txt', 'r') as f:
    for line in f:
        test.preprocessing(line, is_encode=True)
        

In [24]:
my_decoder = MyDecoder(train.counter, test.counter)

Сравним оригинальный текст и расшифрованный

In [25]:
for line_cl, line_en in zip(test.total_clean_text, test.total_encode_text):
    print(line_cl)
    print(my_decoder.get_decode_chars(line_en))
    print()

в лондоне показывали диких зверей и за смотренье брали деньгами или собаками и кошками на корм диким зверям
р иолуола мовегжреин унвнч гракай н ге тпоскалза якеин уалзыепн нин тояевепн н воцвепн ле вокп унвнп гракбп

одному человеку захотелось поглядеть зверей он ухватил на улице собачонку и принес ее в зверинец его пустили смотреть а собачонку взяли и бросили в клетку ко льву на съеденье
оулопд ьаиоравд гечосаиотз моыибуасз гракай ол дчресни ле динюа тояеьолвд н мкнлат аа р гракнлаю аыо мдтснин тпоскасз е тояеьолвд ргбин н якотнин р виасвд во изрд ле тщауалза

собачка поджала хвост и прижалась в угол клетки лев подошел к ней и понюхал ее
тояеьве моухеие чротс н мкнхеиетз р дыои виасвн иар моуоцаи в лай н молшчеи аа

собачка легла на спину подняла лапки и стала махать хвостиком
тояеьве иаыие ле тмнлд моулбие иемвн н тсеие печесз чротснвоп

лев тронул ее лапой и перевернул
иар сколди аа иемой н макараклди

собачка вскочила и стала перед львом на задние лапки
тояеьве ртвоьние н тсеие мак

Получилось как-то не очень

# 2
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм.


In [26]:
class BigramFreq():
    def __init__(self):
        self.counter = Counter()
        self.count_bigram = 0
        self.total_encode_text = []
        self.total_clean_text = []
        
    def _cleaning_text(self, line, is_encode):
        encode_text = ''
        clean_line = ''
        for char in line:
            if char not in PUNCT:
                clean_line += char
                if is_encode:
                    char = encoder(char, code)
                    encode_text += char
        if clean_line:
            self.total_clean_text.append(clean_line)
        if encode_text:
            self.total_encode_text.append(encode_text)
            
    def _get_bigramm_count(self):
        encoded_line = ''
        for line in self.total_clean_text:
            line = ' ' + line
            for i in range(len(line) - 1):
                self.counter[line[i] + line[i + 1]] += 1
                self.count_bigram += 1
                
    def preprocessing(self, line, is_encode=False):
        line = line.strip().lower()
        self._cleaning_text(line, is_encode=is_encode)
        self._get_bigramm_count()
        self._sort_chars()
        
    def _sort_chars(self):
        self.counter = Counter({k: v for k, v in sorted(self.counter.items(), key=lambda item: -item[1])})
        
            

In [27]:
train2 = BigramFreq()
with open('WarAndPeace.txt', 'r') as f:
    all_text = f.read()
    
all_text = all_text.replace('\n', ' ')

In [28]:
train2.preprocessing(all_text)

In [29]:
test2 = BigramFreq()

with open('LionAnd_Dog.txt', 'r') as f:
    test_text = f.read()

test_text = test_text.replace('\n', ' ')
test2.preprocessing(test_text, is_encode=True)

In [30]:
decoder_bigramm = MyDecoder(train2.counter, test2.counter)

Посмотрим на расшифрованный текст

In [31]:
print(decoder_bigramm.get_decode_bigrams(test2.total_encode_text))

ъьрлнилнйьшлдяхфъярмьимдмчьхъйбйаьмьхяьвелэбйн йьпбярмьийн сяемьмрмьвлпядяемьмьдлтдяемьняьдлбеьимдмеьхъйбоеьлинлегьцйрлъйдгьхячлэйрлв ьшлсроийэ ьхъйбйаьлньгчъяэмрьняьгрмюйьвлпяцлндгьмьшбмнйвьййьъьхъйбмнйюьйсльшгвэмрмьвелэбйэ ьяьвлпяцлндгьъхормьмьпблвмрмьъьдрйэдгьдльр ъгьняьвжйийн йьвлпяцдяьшлиуяряьчълвэьмьшбмуяряв ьъьгслрьдрйэдмьрйъьшлилтйрьдьнйаьмьшлнкчярьййьвлпяцдяьрйсряьняьвшмнгьшлиноряьряшдмьмьвэяряьеячяэ ьчълвэмдлеьрйъьэблнгрьййьряшлаьмьшйбйъйбнгрьвлпяцдяьъвдлцмряьмьвэяряьшйбйиьр ълеьняьхяинмйьряшдмьрйъьвелэбйрьняьвлпяцдгьшлълбяцмъярьслрлъгьвльвэлблнфьняьвэлблнгьмьнйьэблсярьййьдлсияьчлхомньпблвмрьр ъгьеовяьрйъьлэлбъярьдгвлдьмьлвэяъмрьвлпяцдйьъйцйблеьдлсияьрйъьрйсьвшяэ ьвлпяцдяьрйсряьшлирйьнйсльмьшлрлумряьвълкьслрлъгьйегьняьряшгьвьэйчьшлбьвлпяцдяьумряьъьлинлаьдрйэдйьвльр ълеьрйъьнйьэблсярьййьйрьдлбеьвшярьвьнйаьъейвэйьяьмнлсияьмсбярьвьнйаьлимньбяхьпябмньшбмтйрьъьхъйбмнйюьмьгхнярьвълкьвлпяцдгьлньвдяхярьцэльвлпяцдяьйсльвлпвэъйнняоьмьшлшблвмрьчлхомняьхъйбмнюяьлэияэ ьйегьчлхомньчлэйрьлэ

Получилось что-то совсем нечитаемое

# 3
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Воспользуемся алгоритмом Метрополиса-Гастингса. Требуется расшифровать сообщение, т.е. найти нашу перестановку символов алфавита. На каждом шаге будем делать случайную перестановку символов (начинаем с тождественной перестановки), с помощью полученного ключа расшифровывать сообщение. Решение о том принимать перестановку или нет будем принимать на основе функции правдоподобия (которая получается из статистики биграмм).
* Если правдоподобие больше предыдущего, тогда перестановку принимаем
* Если меньше, перестановку принимаем с вероятностью $\dfrac{NewLikelihood}{OldLikelihood}$

In [32]:
def log_likelihood(text, train_freq_counter):
    res = 0
    for i in range(len(text) - 1):
        bigram = text[i] + text[i + 1]
        prob = train_freq_counter.counter.get(bigram, EPS) / train_freq_counter.count_bigram
        res += np.log(prob)
    return res

In [33]:
def metropolis_hastings_log_accept(l, l_new):
    if l_new > l:
        return True
    else:
        return (np.random.rand() < (np.exp(l_new - l)))

In [34]:
def metropolis_hastings(encoded_text, train_freq_counter, n_steps=10000, verbose=True, log_step=1000):
    cur_code = list(range(N_SIMBOLS))
    cur_text = encoded_text
    cur_likelihood = log_likelihood(cur_text, train_freq_counter)
    for step in range(n_steps):
        i, j = np.random.choice(N_SIMBOLS, 2, replace=False)
        cur_code[i], cur_code[j] = cur_code[j], cur_code[i]
        new_text = decoding_text(encoded_text, cur_code)
        if cur_text != new_text:
            new_likelihood = log_likelihood(new_text, train_freq_counter)
            if metropolis_hastings_log_accept(cur_likelihood, new_likelihood):
                cur_text = new_text
                cur_likelihood = new_likelihood
            else:
                cur_code[i], cur_code[j] = cur_code[j], cur_code[i]
        else:
            cur_code[i], cur_code[j] = cur_code[j], cur_code[i]
        if verbose:
            if step and step % log_step == 0:
                print(f'Step: {step}')
                print(cur_text)
                print()
    if verbose:
        print(f'Step: {step}')
        print(cur_text)
    return cur_text

Оригинальный текст

In [35]:
test2.total_clean_text[0]

'в лондоне показывали диких зверей и за смотренье брали деньгами или собаками и кошками на корм диким зверям одному человеку захотелось поглядеть зверей он ухватил на улице собачонку и принес ее в зверинец его пустили смотреть а собачонку взяли и бросили в клетку ко льву на съеденье собачка поджала хвост и прижалась в угол клетки лев подошел к ней и понюхал ее собачка легла на спину подняла лапки и стала махать хвостиком лев тронул ее лапой и перевернул собачка вскочила и стала перед львом на задние лапки лев смотрел на собачку поворачивал голову со стороны на сторону и не трогал ее когда хозяин бросил льву мяса лев оторвал кусок и оставил собачке вечером когда лев лег спать собачка легла подле него и положила свою голову ему на лапу с тех пор собачка жила в одной клетке со львом лев не трогал ее ел корм спал с ней вместе а иногда играл с ней один раз барин пришел в зверинец и узнал свою собачку он сказал что собачка его собственная и попросил хозяина зверинца отдать ему хозяин хотел о

Зашифрованный текст

In [36]:
encoded_text = test2.total_encode_text[0]
encoded_text

'ъьрлнилнйьшлдяхфъярмьимдмчьхъйбйаьмьхяьвелэбйн йьпбярмьийн сяемьмрмьвлпядяемьмьдлтдяемьняьдлбеьимдмеьхъйбоеьлинлегьцйрлъйдгьхячлэйрлв ьшлсроийэ ьхъйбйаьлньгчъяэмрьняьгрмюйьвлпяцлндгьмьшбмнйвьййьъьхъйбмнйюьйсльшгвэмрмьвелэбйэ ьяьвлпяцлндгьъхормьмьпблвмрмьъьдрйэдгьдльр ъгьняьвжйийн йьвлпяцдяьшлиуяряьчълвэьмьшбмуяряв ьъьгслрьдрйэдмьрйъьшлилтйрьдьнйаьмьшлнкчярьййьвлпяцдяьрйсряьняьвшмнгьшлиноряьряшдмьмьвэяряьеячяэ ьчълвэмдлеьрйъьэблнгрьййьряшлаьмьшйбйъйбнгрьвлпяцдяьъвдлцмряьмьвэяряьшйбйиьр ълеьняьхяинмйьряшдмьрйъьвелэбйрьняьвлпяцдгьшлълбяцмъярьслрлъгьвльвэлблнфьняьвэлблнгьмьнйьэблсярьййьдлсияьчлхомньпблвмрьр ъгьеовяьрйъьлэлбъярьдгвлдьмьлвэяъмрьвлпяцдйьъйцйблеьдлсияьрйъьрйсьвшяэ ьвлпяцдяьрйсряьшлирйьнйсльмьшлрлумряьвълкьслрлъгьйегьняьряшгьвьэйчьшлбьвлпяцдяьумряьъьлинлаьдрйэдйьвльр ълеьрйъьнйьэблсярьййьйрьдлбеьвшярьвьнйаьъейвэйьяьмнлсияьмсбярьвьнйаьлимньбяхьпябмньшбмтйрьъьхъйбмнйюьмьгхнярьвълкьвлпяцдгьлньвдяхярьцэльвлпяцдяьйсльвлпвэъйнняоьмьшлшблвмрьчлхомняьхъйбмнюяьлэияэ ьйегьчлхомньчлэйрьл

In [40]:
decoded_text = metropolis_hastings(encoded_text, train2, n_steps=10000, log_step=1000)

Step: 1000
н тедредо бевазьнати ривия знолою и за спеклодцо млати родцхапи ити семавапи и вешвапи да велп ривип знолып ердепу готенову заяекотесц бехтырокц знолою ед уянакит да утифо семагедву и блидос оо н знолидоф охе бускити спеклокц а семагедву нзыти и млесити н втокву ве тцну да съородцо семагва берщата янеск и блищатасц н ухет втокви тон берешот в дою и беджяат оо семагва тохта да сбиду бердыта табви и ската паяакц янескивеп тон кледут оо табею и болонолдут семагва нсвегита и ската болор тцнеп да зардио табви тон спеклот да семагву бенелагинат хетену се скеледь да скеледу и до клехат оо вехра яезыид млесит тцну пыса тон екелнат вусев и есканит семагво ноголеп вехра тон тох сбакц семагва тохта берто дохе и бетещита снеж хетену опу да табу с коя бел семагва щита н ердею втокво се тцнеп тон до клехат оо от велп сбат с дою нпоско а идехра ихлат с дою ерид лаз малид блишот н знолидоф и уздат снеж семагву ед свазат гке семагва охе семскноддаы и беблесит яезыида знолидфа екракц опу яезы

Step: 6000
в лондоне показывали диких зверей и за смотренье брали деньгами или собаками и кожками на корм диким зверям одному человеку захотелось поглядеть зверей он ухватил на улице собачонку и принес ее в зверинец его пустили смотреть а собачонку взяли и бросили в клетку ко льву на съеденье собачка подшала хвост и пришалась в угол клетки лев подожел к ней и понюхал ее собачка легла на спину подняла лапки и стала махать хвостиком лев тронул ее лапой и перевернул собачка вскочила и стала перед львом на задние лапки лев смотрел на собачку поворачивал голову со стороны на сторону и не трогал ее когда хозяин бросил льву мяса лев оторвал кусок и оставил собачке вечером когда лев лег спать собачка легла подле него и полошила свою голову ему на лапу с тех пор собачка шила в одной клетке со львом лев не трогал ее ел корм спал с ней вместе а иногда играл с ней один раз барин прижел в зверинец и узнал свою собачку он сказал что собачка его собственная и попросил хозяина зверинца отдать ему хозя

In [41]:
sucsess = 0
for c1, c2 in zip(test2.total_clean_text[0], decoded_text):
    if c1 == c2:
        sucsess += 1
print(f'Тексты совпадают на {sucsess / len(decoded_text) * 100:.2f}%')

Тексты совпадают на 99.38%


Получился очень хороший результат.

# 4. Расшифруйте сообщение:

დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

Так как сообщение состоит не из символов алфавита, необходимо сделать соответствие старого и нового алфавита.

In [42]:
test_task = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [43]:
task_simbols = set(test_task)

In [44]:
true_simbols = train.counter.keys()

In [45]:
mapping = {k: v for k, v in zip(list(task_simbols), list(true_simbols))}

In [46]:
new_test_task = ''
for simbol in test_task:
    new_test_task += mapping[simbol]

Посмотрим на зашифрованное сообщение, но закодированное уже русскими буквами

In [47]:
new_test_task

'уршмьдпьдмямыуьанбгтшсапжьмшмь нчымьанбгтшсапжьыукрыьзьюыниньрннхоуамвькнынбпжьшуикнь бнчмытысьркнбууьдруиньдпьдруьряуштшмь бтдмшсаньмь ншзчмыуьгткрмгтшсапжьхтшшьеть нршуяаууьчуыдубынуьетятамуькзбртьлнывькнаучаньвьамчуиньауьнхуотй'

In [55]:
decoded_test_task = metropolis_hastings(new_test_task, train2, n_steps=100000, log_step=10000)

Step: 10000
если вы вимите норзальный или подти норзальный текст у чтого сообщения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный балл жа послемнее детвертое жамание курса хотя конедно я нидего не обещаш

Step: 20000
если вы вимите норзальный или почти норзальный текст у этого соодшения который легко прочитать скорее всего вы все смелали правильно и получите заксизальный далл ца послемнее четвертое цамание курса хотя конечно я ничего не одешаф

Step: 30000
если вы вимите норжальный или подти норжальный текст у этого сообчения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный балл за послемнее детвертое замание курса шотя конедно я нидего не обечаю

Step: 40000
если вы вимите норзальный или подти норзальный текст у этого сообщения который легко продитать скорее всего вы все смелали правильно и полудите заксизальный балл ча послемнее детвертое чамание курса хотя конедно я нидего не обещаж

Step: 50000
если вы вими

In [56]:
decoded_test_task

'если вы вимите норжальный или подти норжальный текст у этого сообчения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный балл за послемнее детвертое замание курса хотя конедно я нидего не обечаш'

Полагаю, что исходный текст был такой:  
***если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю***

In [60]:
true_task_text = 'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю'

In [61]:
sucsess = 0
for c1, c2 in zip(true_task_text, decoded_test_task):
    if c1 == c2:
        sucsess += 1
print(f'Тексты совпадают на {sucsess / len(decoded_test_task) * 100:.2f}%')

Тексты совпадают на 92.61%
